In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import cv2
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = r'/content/drive/My Drive/trimmed_waveform_1'
csv_filename = 'file_status.csv'

# Get list of files in the directory
files = os.listdir(path)

# Create CSV file and write headers
with open(csv_filename, 'w', newline='') as csvfile:
    fieldnames = ['real_value']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    # Write 1 if filename starts with 'L', otherwise write 0

    for i in glob.glob(path + "/L*.npy"):
      writer.writerow({'real_value': 1})
    for i in glob.glob(path + "/*.npy"):
      writer.writerow({'real_value': 0})

print("CSV file created successfully.")



CSV file created successfully.


In [ ]:
lensed_waveform = []  # Initialize lists to store filenames
unlensed_waveform = []

for i in glob.glob(path + "/L*.npy"):
    lensed_waveform.append(i)

for i in glob.glob(path + "/*.npy"):
    if not any(i == lwf for lwf in lensed_waveform):
        unlensed_waveform.append(i)

In [ ]:
len(lensed_waveform)

10000

In [ ]:
len(unlensed_waveform)

10000

In [ ]:
lensed_waveform = [np.load(file) for file in lensed_waveform]
unlensed_waveform = [np.load(file) for file in unlensed_waveform]

In [ ]:
desired_shape = (614, 1)
lensed_waveform = [np.resize(waveform, desired_shape) for waveform in lensed_waveform]
unlensed_waveform = [np.resize(waveform, desired_shape) for waveform in unlensed_waveform]

In [ ]:
# Created labels
lensed_labels = np.ones(len(lensed_waveform))
unlensed_labels = np.zeros(len(unlensed_waveform))

In [ ]:
# Combine data and labels
import numpy as np
import tensorflow as tf
np.random.seed(42)
tf.random.set_seed(42)

from sklearn.model_selection import train_test_split
X = np.concatenate([lensed_waveform, unlensed_waveform])
y = np.concatenate([lensed_labels, unlensed_labels])

# Split data into training and testing sets
# from sklearn.model_selection import train_test_split

# Assuming your data is in X and y variables
# X represents the features, and y represents the target variable

# Split the data into training and testing sets (80% training, 20% test+validation)
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the test+validation set into test and validation sets (50% test, 50% validation)
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [ ]:
import numpy as np
import csv
from keras import models, layers
from keras.optimizers import Adam

# Assuming X_train, X_val, y_train, y_val, X_test, y_test are already defined

# Function to classify files
def classify_file(model, data):
    # Predict the class using the model
    prediction = model.predict(data)

    # Round the prediction to get binary classification result (0 or 1)
    return np.round(prediction).astype(int)

# Load the trained model
model = models.Sequential()
model.add(layers.Input(shape=(614, 1)))
model.add(layers.Conv1D(filters=256, kernel_size=2, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Conv1D(filters=256, kernel_size=2, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Conv1D(filters=128, kernel_size=2, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Conv1D(filters=128, kernel_size=2, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Conv1D(filters=64, kernel_size=2, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Conv1D(filters=64, kernel_size=2, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Conv1D(filters=64, kernel_size=2, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(7656,activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(lr=0.00001), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val))
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Predictions for test data
test_predictions = classify_file(model, X_test)

# Create CSV file and write predictions
csv_filename = 'final_data.csv'
with open(csv_filename, 'w', newline='') as csvfile:
    fieldnames = ['real_value', 'pred_data']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    # Write 0 for the first 10000 files
   # for pred in test_predictions:
     # writer.writerow({'real_value': 0,'pred_data': pred })

    # Write predictions for the last 10000 files
       # for pred in test_predictions:
          #  writer.writerow({'pred_data': pred})

           # writer.writerow({'real_value': 1, 'pred_data': pred})


    # Write predictions for the last 10000 files
    for pred in test_predictions:
        writer.writerow({'pred_data': pred})

print("CSV file created successfully.")


Epoch 1/30
500/500 [==============================] - 528s 1s/step - loss: 0.6915 - accuracy: 0.5056 - val_loss: 0.6932 - val_accuracy: 0.4805
Epoch 2/30
500/500 [==============================] - 513s 1s/step - loss: 0.6932 - accuracy: 0.5023 - val_loss: 0.6942 - val_accuracy: 0.4805
Epoch 3/30
500/500 [==============================] - 508s 1s/step - loss: 0.6932 - accuracy: 0.5013 - val_loss: 0.6932 - val_accuracy: 0.4805
Epoch 4/30
500/500 [==============================] - 503s 1s/step - loss: 0.6932 - accuracy: 0.4973 - val_loss: 0.6932 - val_accuracy: 0.4805
Epoch 5/30
500/500 [==============================] - 498s 996ms/step - loss: 0.6932 - accuracy: 0.4923 - val_loss: 0.6932 - val_accuracy: 0.4805
Epoch 6/30
500/500 [==============================] - 500s 1s/step - loss: 0.6932 - accuracy: 0.5008 - val_loss: 0.6934 - val_accuracy: 0.4805
Epoch 7/30
500/500 [==============================] - 503s 1s/step - loss: 0.6932 - accuracy: 0.4973 - val_loss: 0.6932 - val_accuracy: 0.4

In [ ]:
import csv

# Open the CSV file for reading
with open('final_data.csv', 'r', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    reader = csv.DictReader(csvfile)
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for row in reader:
        # Convert string values to integers
        real_value = bool(row['real_value'])
        pred_data = bool(row['pred_data'])


        # Increment tp, tn, fp, or fn based on conditions

        if real_value == 1 and pred_data == True:
            tp += 1
        elif real_value == 0 and pred_data == False:
            tn += 1
        elif real_value == 0 and pred_data == True:
            fp += 1
        elif real_value == 1 and pred_data == False:
            fn += 1


print("True Positives (TP):", tp)
print("True Negatives (TN):", tn)
print("False Positives (FP):", fp)
print("False Negatives (FN):", fn)


True Positives (TP): 19990
True Negatives (TN): 1
False Positives (FP): 0
False Negatives (FN): 9


In [ ]:
print("accuracy=", (tp+tn)*100/float(tp+tn+fn+fp))
print("recall=", tp*100/float(tp+fn))

accuracy= 99.955
recall= 99.9549977498875


In [ ]:
precision = (tp*100)/float(tp+fp);
recall= tp*100/float(tp+fn)
f1_score = (2*precision*recall)/(precision+recall)
print("f1_score=",f1_score)

f1_score= 99.97749381079797
